In [5]:
using Statistics: mean  # standard library
function loss_and_accuracy(model, data)
    (x,y) = only(loader(data; batchsize=length(data)))
    ŷ = model(x)
    loss = Flux.logitcrossentropy(ŷ, y)  # did not include softmax in the model
    acc = round(100 * mean(Flux.onecold(ŷ) .== Flux.onecold(y)); digits=2)
    (; loss, acc, split=data.split)  # return a NamedTuple
end

loss_and_accuracy (generic function with 1 method)

In [11]:
# Opracowane na podstawie https://github.com/FluxML/model-zoo/blob/3e91af32ebfad628b616618b11bfff2f9f519bec/vision/conv_mnist/conv_mnist.jl
using MLDatasets, Flux
train_data = MLDatasets.MNIST(split=:train)
test_data  = MLDatasets.MNIST(split=:test)

function loader(data; batchsize::Int=1)
    x4dim = reshape(data.features, 28, 28, 1, :) # insert trivial channel dim
    yhot  = Flux.onehotbatch(data.targets, 0:9)  # make a 10×60000 OneHotMatrix
    Flux.DataLoader((x4dim, yhot); batchsize, shuffle=true)
end

net = Chain(
    Conv((3, 3), 1 => 6,  relu),
    MaxPool((2, 2)),
    Conv((3, 3), 6 => 16, relu),
    MaxPool((2, 2)),
    Flux.flatten,
    Dense(400 => 84, relu), 
    Dense(84 => 10, identity),
)

#= ew. prostsza architektura:

net = Chain(
    Conv((3, 3), 1 => 6,  relu, bias=false),
    MaxPool((2, 2)),
    Flux.flatten,
    Dense(13*13*6 => 84, relu, bias=false), 
    Dense(84 => 10, identity, bias=false)
)
=#

Chain(
  Conv((3, 3), 1 => 6, relu),           # 60 parameters
  MaxPool((2, 2)),
  Conv((3, 3), 6 => 16, relu),          # 880 parameters
  MaxPool((2, 2)),
  Flux.flatten,
  Dense(400 => 84, relu),               # 33_684 parameters
  Dense(84 => 10),                      # 850 parameters
)                   # Total: 8 arrays, 35_474 parameters, 139.773 KiB.

In [2]:
train_data[1]

(features = Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], targets = 5)

In [6]:
x1, y1 = first(loader(train_data)); # (28×28×1×1 Array{Float32, 3}, 10×1 OneHotMatrix(::Vector{UInt32}))
y1hat = net(x1)
@show hcat(Flux.onecold(y1hat, 0:9), Flux.onecold(y1, 0:9))

@show loss_and_accuracy(net, test_data);  # accuracy about 10%, before training

train_log = []
settings = (;
    eta = 1e-2,
    epochs = 3,
    batchsize = 100,
)

opt_state = Flux.setup(Descent(settings.eta), net);

hcat(Flux.onecold(y1hat, 0:9), Flux.onecold(y1, 0:9)) = [3 4]
loss_and_accuracy(net, test_data) = (loss = 2.3213522f0, acc = 14.88, split = :test)


In [7]:
x1

28×28×1×1 Array{Float32, 4}:
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0       …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0       …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮              ⋱                      ⋮         
 0.0  0.0  0.0  0.0  0.0  0.0  0.760784     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.

In [12]:
for epoch in 1:settings.epochs
    @time for (x,y) in loader(train_data, batchsize=settings.batchsize)
        #grads = Flux.gradient(model -> Flux.logitcrossentropy(model(x), y), net)
        #Flux.update!(opt_state, net, grads[1])
    end
    
    loss, acc, _ = loss_and_accuracy(net, train_data)
    test_loss, test_acc, _ = loss_and_accuracy(net, test_data)
    @info epoch acc test_acc
    nt = (; epoch, loss, acc, test_loss, test_acc) 
    push!(train_log, nt)
end

  0.331442 seconds (13.61 k allocations: 181.881 MiB, 2.31% compilation time)


┌ Info: 1
│   acc = 17.61
│   test_acc = 17.38
└ @ Main d:\Projects\CNNLibrary\reference_solution.ipynb:9


  0.381864 seconds (11.53 k allocations: 181.744 MiB)
  0.124086 seconds (11.53 k allocations: 181.744 MiB, 19.03% gc time)


┌ Info: 2
│   acc = 17.61
│   test_acc = 17.38
└ @ Main d:\Projects\CNNLibrary\reference_solution.ipynb:9
┌ Info: 3
│   acc = 17.61
│   test_acc = 17.38
└ @ Main d:\Projects\CNNLibrary\reference_solution.ipynb:9


In [9]:
train_log

3-element Vector{Any}:
 (epoch = 1, loss = 0.37241438f0, acc = 88.82, test_loss = 0.35812196f0, test_acc = 89.18)
 (epoch = 2, loss = 0.24756914f0, acc = 92.48, test_loss = 0.23381253f0, test_acc = 92.93)
 (epoch = 3, loss = 0.19051741f0, acc = 94.27, test_loss = 0.1765097f0, test_acc = 94.71)